# Molecular properties
## Structure optimization

One of the most commonly calculated properties in theoretical chemistry is the molecular structure. It is quite common to use structures calculated at a cheaper level of theory, typically DFT, as a starting point since ground state energies are relatively insensitive to the geometry. However, in some cases, especially when strong correlation is significant, one may want to use MCSCF structures instead.

The workflow here is very similar to that of VeloxChem, namely first defining the Gradient Driver, and then give it to the optimizer, which itself is based on GeomeTRIC. The MCSCF gradient driver will automatically choose analytical gradients if available for the specific type of MCSCF calculation performed, numerical gradients otherwise.

```{note}
Currently, derivative integrals are not available and all gradients are therefore numerical. This should change very soon.
```

Let's illustrate this on a simple O$_2$ molecule. First, we compute one MCSCF with the $\pi$ orbitals in the active space.

In [1]:
import veloxchem as vlx
import multipsi as mtp

O2_xyz="""2

O 0.0 0.0 -0.6
O 0.0 0.0  0.6
"""
molecule=vlx.Molecule.from_xyz_string(O2_xyz)
molecule.set_multiplicity(3)
basis = vlx.MolecularBasis.read(molecule,"cc-pvdz")

scfdrv = vlx.ScfUnrestrictedDriver()
scfdrv.compute(molecule, basis)
uhf_orbs = scfdrv.natural_orbitals()

space=mtp.OrbSpace(molecule,uhf_orbs)
space.CAS(6,4)
mcscfdrv=mtp.McscfDriver()
mcscfdrv.compute(molecule,basis,space)

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 4.
                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Unrestricted Hartree-Fock                                       
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error


          Active space definition:
          ------------------------
Number of inactive (occupied) orbitals: 5
Number of active orbitals:              4
Number of virtual orbitals:             19

    This is a CASSCF wavefunction: CAS(6,4)

          CI expansion:
          -------------
Number of determinants:      6


                                                                                                                          
        MCSCF Iterations
        ----------------
                                                                                                                          
     Iter. | Average Energy | E. Change | Grad. Norm | CI Iter. |   Time
     ---------------------------------------------------------------------
        1     -149.645938143     0.0e+00      5.7e-02          1   0:00:00
        2     -149.646675896    -7.4e-04      1.8e-02          1   0:00:00
        3     -149.646744172    -6.8e-05      4.0e-03          1   0:00:00
      

                                                                                                                          
Total MCSCF time: 00:00:00


Now, we optimize the molecule.

In [2]:
GradDrv=mtp.McscfGradientDriver(mcscfdrv)
opt_drv = vlx.OptimizationDriver(GradDrv)
casscf_opt_geom = opt_drv.compute(molecule, basis, space)

                                                                                                                          
                                                Optimization Driver Setup                                                 
                                                                                                                          
                                     Coordinate System       :    TRIC                                                    
                                     Constraints             :    No                                                      
                                     Max. Number of Steps    :    300                                                     
                                                                                                                          
* Info * Computing energy and gradient...                                                                                 
* Info *   Energ

We can now use this new geometry in subsequent calculations.

In [3]:
molecule=casscf_opt_geom
mcscfdrv.compute(molecule,basis,space)


          Active space definition:
          ------------------------
Number of inactive (occupied) orbitals: 5
Number of active orbitals:              4
Number of virtual orbitals:             19

    This is a CASSCF wavefunction: CAS(6,4)

          CI expansion:
          -------------
Number of determinants:      6


                                                                                                                          
        MCSCF Iterations
        ----------------
                                                                                                                          
     Iter. | Average Energy | E. Change | Grad. Norm | CI Iter. |   Time
     ---------------------------------------------------------------------
        1     -149.647278689     0.0e+00      9.8e-06          1   0:00:00
                                                                                                                          
** Convergence reached in 1 itera

                                                                                                                          
Total MCSCF time: 00:00:00


If your MCSCF calculation is a state-averaged, you should extend the parameters of the compute function with the number of states in the state-averaging and the index (starting from 0) of the state you wish to optimize:

```
casscf_opt_geom = opt_drv.compute(molecule, basis, space, nstates, istate)
```

## Spectroscopy

MCSCF is commonly used to simulate spectroscopy and photochemistry. There are several ways to compute excited state properties from a MCSCF calculation. Currently, MultiPsi implements only properties from a state-averaged calculation.

Let's go back to our furan calculation. First we compute a SA-CASSCF with 5 states:

In [4]:
furan_xyz="""9

 C     -0.86213    -0.90784     0.00007
 H     -1.63433    -1.64264    -0.00003
 C      0.50727    -0.90524     0.00007
 C      0.92057     0.47886    -0.00003
 C     -0.22323     1.23186    -0.00003
 O     -1.35123     0.40376    -0.00013
 H      1.17117    -1.74724     0.00017
 H      1.93767     0.81866     0.00007
 H     -0.46573     2.26986    -0.00013
"""

molecule = vlx.Molecule.from_xyz_string(furan_xyz)
basis = vlx.MolecularBasis.read(molecule,"def2-sv(p)")

#We use a previously stored h5 file for the active space
space=mtp.OrbSpace(molecule,"furan-cas.h5")
mcscfdrv=mtp.McscfDriver()
mcscfdrv.compute(molecule,basis,space, 5) #state-averaged with 5 states


          Active space definition:
          ------------------------
Number of inactive (occupied) orbitals: 15
Number of active orbitals:              5
Number of virtual orbitals:             58

    This is a CASSCF wavefunction: CAS(6,5)

          CI expansion:
          -------------
Number of determinants:      55


                                                                                                                          
        MCSCF Iterations
        ----------------
                                                                                                                          
     Iter. | Average Energy | E. Change | Grad. Norm | CI Iter. |   Time
     ---------------------------------------------------------------------
        1     -228.211383987     0.0e+00      1.6e-01          1   0:00:00
        2     -228.223997559    -1.3e-02      8.1e-02          1   0:00:00
        3     -228.226393803    -2.4e-03      1.8e-02          1   0:00:00
    

                                                                                                                          
Total MCSCF time: 00:00:06


Now we can compute the transition properties using the InterState module. This module requires the CI vectors computed by the MCSCF.

In [5]:
#Compute oscillator strengths
SI=mtp.InterState()
Prop_dict=SI.diposc(molecule,basis,mcscfdrv.CIVecs)

                                                                                                                          
List of oscillator strengths greather than 1e-10
                                                                                                                          
  From     to       Energy (eV)    Oscillator strength (length and velocity)
     1       2        6.66708         3.935228e-03    2.925456e-03
     1       3        8.29230         2.426600e-01    1.672872e-01
     1       5       10.11827         5.706685e-01    4.028088e-01


The function returns a dictionary with the most important values.

In [6]:
print(Prop_dict["energies"])
print(Prop_dict["oscillator_strengths"])

[0.24501075 0.30473646 0.35263678 0.37183941]
[3.93522815e-03 2.42660014e-01 2.14263136e-24 5.70668499e-01]


By default InterState computes properties from the first state to all others, but this can be changed by providing the list of "initial" and "final" states as arguments.

In [7]:
Prop_dict=SI.diposc(molecule,basis,mcscfdrv.CIVecs, ifrom=[0,1], ito='all' )

                                                                                                                          
List of oscillator strengths greather than 1e-10
                                                                                                                          
  From     to       Energy (eV)    Oscillator strength (length and velocity)
     1       2        6.66708         3.935228e-03    2.925456e-03
     1       3        8.29230         2.426600e-01    1.672872e-01
     1       5       10.11827         5.706685e-01    4.028088e-01
     2       3        1.62522         2.610885e-02    2.178061e-02
     2       5        3.45118         1.364278e-03    1.798930e-04


The diposc function is a shortcut to the compute function that takes a list of properties as argument. Thus it is equivalent to calling

```
Prop_dict = SI.compute(molecule,basis, mcscfdrv.CIVecs, ["absorption"], [0,1], 'all')
```

The list of properties can also accept 'ecd' to compute electric circular dichroism.

```
Prop_dict = SI.compute(molecule,basis, mcscfdrv.CIVecs, ["absorption", "ecd"], [0,1], 'all')
```
which adds "rotatory_strengths" to the dictionary. There is also a shortcut function to compute only the ECD spectrum:

```
Prop_dict = SI.ecd(molecule,basis, mcscfdrv.CIVecs)
```